In [ ]:
!pip install transformers datasets -q

Ссылка на датасет: https://huggingface.co/datasets/DGurgurov/azerbaijani_sa/viewer/default/test?views%5B%5D=test

In [ ]:
from datasets import load_dataset

dataset = load_dataset("DGurgurov/azerbaijani_sa")


# Пример с азербайджанскими данными

Данные для примера

Выберем случайные 50 примеров из тестовой выборки. Также задаём переменную sentiment, соответствующую истинным меткам датасета

In [ ]:
samples = dataset["test"].select(range(50))
sentiment = samples["label"]
print(sentiment[1] )

Загрузка токенизатора и модели

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
DEVICE = torch.device("cuda:0")

az_checkpoint = "ai-forever/mGPT-1.3B-azerbaijan"
az_tokenizer = AutoTokenizer.from_pretrained(az_checkpoint)
az_model = AutoModelForCausalLM.from_pretrained(az_checkpoint).to(DEVICE)

Рассмотрим бинарную классификацию по тональности без дообучения.

## Метод 1: few-shot генерация

В ячейке инструкция переведена на азербайджанский. В комментариях я оставил инструкцию по-русски для лучшей ориентации.

Тональность — это эмоциональное отношение автора высказывания к некоторому объекту, выраженное в тексте.
  Она может быть положительной или отрицательной.
  Положительная тональность выражается с помощью метки '1'.
  Отрицательная тональность выражается с помощью метки '0'.

  Ниже приведены примеры классификации текстов по тональности:
  Текст: Этот фильм был ужасен.
  Тональность: 0 (отрицательная)
  Текст: Мне нравится эта книга.
  Тональность: 1 (положительная)
  Текст: Я ненавижу это место.
  Тональность: 0 (отрицательная)
  Текст: Сегодя замечательный день.
  Тональность: 1 (положительная)
  Текст: Я не могу поверить, насколько это здорово!
  Тональность: 1 (положительная)
  Текст: Это самый худший опыт, который у меня когда-либо был.
  Тональность: 0 (отрицательная)
  Текст: {text}
  Тональность:

In [ ]:
def generate_sentiment(text, model, tokenizer):

 # Задаем промпт
  prompt = f"""Tonallıq, ifadə müəllifinin mətndə ifadə olunan bəzi obyektə emosional münasibətidir.
  Müsbət və ya mənfi ola bilər.
  Müsbət tonallıq ' 1 ' işarəsi ilə ifadə olunur.
  Mənfi tonallıq ' 0 ' işarəsi ilə ifadə olunur.

  Aşağıda mətnlərin tonallığa görə təsnif edilməsinə dair nümunələr verilmişdir:
  Mətn: Təssüf ki, Finala qədər gələ bilmədi Finalda Messi Ronaldo görmək istiyərdim.
  Tonallıq: 0 (mənfi)
  Mətn: Və ictimai nəqliyyatdan istifadə etmək asan və rahatdı.
  Tonallıq: 1 (müsbət)
  Mətn: Rusiyanın düşmənlərinin izi ilə getmək, vəziyyəti içəridən sarsıtmaq, mitinqlər keçirmək və hava limanlarını dağıtmaq olmaz.
  Tonallıq: 0 (mənfi)
  Mətn: İlahi. Hər kəsin üzü gülür. Xoşbəxtlik budur.
  Tonallıq: 1 (müsbət)
  Mətn: Və bu sonsuz kainatda dünyamızın, bir toz dənəciyi boyda olduğuna baxmayaraq, insanlar mənasız şeylə görə bir-birini öldürür
  Tonallıq: 0 (mənfi)
  Mətn: Nə gözəl ! Maşallah əksəriyyəti də sağ salamatdırlar . Mənim də orta məktəb məzunu olmağımın 45 illiyi 2025ci ildə tamam olacaq . Kaş bizlər də belə bir şey təşkil edə bilsəydik .
  Tonallıq: 1 (müsbət)
  Mətn: {text}
  Tonallıq:"""

  # Токенизируем текст
  input_ids = tokenizer.encode(prompt, return_tensors="pt").to(DEVICE)

  # Осуществляем генерацию
  out = model.generate(input_ids, do_sample=False, max_new_tokens=1)


  # Декодируем токены
  generated_sentiment = int(list(map(tokenizer.decode, out))[0].split()[-1])
  return generated_sentiment

In [ ]:
predictions = []
for text in samples:
    pred1 = generate_sentiment(text["text"], az_model, az_tokenizer)
    #pred = generate_sentiment(text["text"], pipe)
    print(pred1)
    print()
    predictions.append(pred1)
print(predictions)

Подсчёт метрик:

Точность

In [ ]:
import numpy as np
from sklearn.metrics import precision_score
precision_score( sentiment, predictions)

Полнота

In [ ]:
from sklearn.metrics import recall_score
recall_score(sentiment, predictions)

f1_мера

In [ ]:
from sklearn.metrics import f1_score
f1_score(sentiment,predictions)

Матрица ошибок:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Создание матрицы ошибок
cm = confusion_matrix(sentiment,predictions)

# Визуализация с помощью Seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1], yticklabels=[1, 0])
plt.xlabel('Предсказанные метки')
plt.ylabel('Истинные метки')
plt.title('Матрица ошибок (confusion matrix)')
plt.show()


## Метод 2: использование перплексии

Для определения тональности будем использовать меру перплексии. В теории информации, перплексия – мера того, насколько хорошо распределение вероятностей предсказывает выборку. Говоря простым языком, перплексия – мера "удивлённости" модели.

На вход модели мы подаём затравку в формате <метка тональности> + <отзыв>. Далее мы считаем перплексию каждого из них. Выбираем наименьшую перплексию из двух и присваиваем метку отзыву.

Müsbət tonallıq- положительная тональность

Mənfi tonallıq- отрицательная тональность

In [ ]:
import math

def calculate_perplexity(sentence, model, tokenizer):
    # Добавляем к предложению метки тональности
    sentence_positive = 'Müsbət tonallıq:'+sentence
    sentence_negative = 'Mənfi tonallıq:'+sentence
    list_sent = [sentence_positive, sentence_negative]
    ppl_values = []

    for sentence in list_sent:
      # Токенизируем предложения
      encodings = tokenizer(sentence, return_tensors='pt')
      input_ids = encodings.input_ids.to(DEVICE)
      # Применяем модель
      with torch.no_grad():
          outputs = model(input_ids=input_ids, labels=input_ids)
      loss = outputs.loss
      # Подсчитываем перплексию
      ppl = math.exp(loss.item() * input_ids.size(1))
      ppl_values.append(ppl)

    # Выбираем предложение с меньшим значением перплексии
    if ppl_values[0] > ppl_values[1]:
      return 0 # отрицательная тональность
    else:
      return 1 # положительная тональность

Mətn-текст

In [ ]:
pred2 = []
for text in samples:
    ppl_value = calculate_perplexity(text["text"], az_model, az_tokenizer)
    print(ppl_value)
    print()
    pred2.append(ppl_value)
print(pred2)
 # print(f'Mətn "{text}": {calculate_perplexity(text["text"], az_model, az_tokenizer)}')

Точность

In [ ]:
import numpy as np
from sklearn.metrics import precision_score
precision_score( sentiment, pred2 )

полнота:

In [ ]:
from sklearn.metrics import recall_score
recall_score(sentiment, pred2 )

f1_мера

In [ ]:
from sklearn.metrics import f1_score
f1_score(sentiment, pred2 )

Матрица Ошибок:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Создание матрицы ошибок
cm = confusion_matrix(sentiment,pred2)

# Визуализация с помощью Seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1], yticklabels=[1, 0])
plt.xlabel('Предсказанные метки')
plt.ylabel('Истинные метки')
plt.title('Матрица ошибок (confusion matrix)')
plt.show()


# Применение предобучееной модели Bert к этим же 50 предложениям.

Ссылка на модель: https://huggingface.co/DGurgurov/xlm-r_azerbaijani_sentiment

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

model_name = "DGurgurov/xlm-r_azerbaijani_sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
def analyze_sentiment(text):
    # Токенизация входного текста
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Получение логитов
    with torch.no_grad():
        logits = model(**inputs).logits

    # Получение предсказаний
    predicted_class = torch.argmax(logits, dim=-1).item()


    return predicted_class

In [ ]:
pred3 = []
for text in samples:
    predicted_class = analyze_sentiment(text["text"])
    print(predicted_class)
    print()
    pred3.append(predicted_class)
print(pred3)

Подсчёт Метрик

Точность

In [ ]:
import numpy as np
from sklearn.metrics import precision_score
precision_score( sentiment, pred3 )

Полнота

In [ ]:
from sklearn.metrics import recall_score
recall_score(sentiment, pred3 )

f1_мера

In [ ]:
from sklearn.metrics import f1_score
f1_score(sentiment, pred3 )

Матрица ошибок:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Создание матрицы ошибок
cm = confusion_matrix(sentiment,pred3)

# Визуализация с помощью Seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1], yticklabels=[1, 0])
plt.xlabel('Предсказанные метки')
plt.ylabel('Истинные метки')
plt.title('Матрица ошибок (confusion matrix)')
plt.show()
